<a href="https://colab.research.google.com/github/Hramchenko/Handwritting/blob/master/colab_install_deps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip3 install torch torchvision
!pip3 install opencv-python
import torch

    100% |████████████████████████████████| 2.0MB 10.8MB/s 
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
fastai 1.0.46 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [3]:
print("Device " + torch.cuda.get_device_name(0))
device = torch.device("cuda:0")
print(device)

Device Tesla K80
cuda:0


In [5]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [6]:
!ls /gdrive/My\ Drive/ML

HW_mod.ipynb  IAM  Project_1  Project_docs  utils.py


In [0]:
!mkdir ./IAM
!cp -R -f /gdrive/My\ Drive/ML/IAM/ascii/ ./IAM/ascii
!cp -f /gdrive/My\ Drive/ML/IAM/words.test.pkl ./IAM/
!cp -f /gdrive/My\ Drive/ML/IAM/words.train.pkl ./IAM/


In [8]:
!git clone https://github.com/Hramchenko/Handwritting.git

Cloning into 'Handwritting'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 32 (delta 12), reused 18 (delta 5), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [9]:
!cd ./Handwritting/ && git pull 

Already up to date.


In [0]:
import sys
sys.path.append("./Handwritting/")

In [11]:
from IAMWords import IAMWords
words = IAMWords("train", "./IAM/")

Reading ./IAM/words.train.pkl...
Reading finished


In [12]:
!git clone https://github.com/SeanNaren/warp-ctc.git;\
cd warp-ctc;\
mkdir build;\
cd build;\
cmake ..;\
make;

!cd ./warp-ctc/pytorch_binding;\
python setup.py install

Cloning into 'warp-ctc'...
remote: Enumerating objects: 751, done.
remote: Total 751 (delta 0), reused 0 (delta 0), pack-reused 751
Receiving objects: 100% (751/751), 374.87 KiB | 16.30 MiB/s, done.
Resolving deltas: 100% (378/378), done.
-- The C compiler identification is GNU 7.3.0
-- The CXX compiler identification is GNU 7.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthr

In [3]:
! python -m site

sys.path = [
    '/content',
    '/env/python',
    '/usr/lib/python36.zip',
    '/usr/lib/python3.6',
    '/usr/lib/python3.6/lib-dynload',
    '/usr/local/lib/python3.6/dist-packages',
    '/usr/local/lib/python3.6/dist-packages/warpctc_pytorch-0.1-py3.6-linux-x86_64.egg',
    '/usr/lib/python3/dist-packages',
]
USER_BASE: '/root/.local' (exists)
USER_SITE: '/root/.local/lib/python3.6/site-packages' (doesn't exist)
ENABLE_USER_SITE: True


In [0]:
import torch
from warpctc_pytorch import CTCLoss
ctc_loss = CTCLoss()
# expected shape of seqLength x batchSize x alphabet_size
probs = torch.FloatTensor([[[0.1, 0.6, 0.1, 0.1, 0.1], [0.1, 0.1, 0.6, 0.1, 0.1]]]).transpose(0, 1).contiguous()
labels = torch.IntTensor([1, 2])
label_sizes = torch.IntTensor([2])
probs_sizes = torch.IntTensor([2])
probs.requires_grad_(True)  # tells autograd to compute gradients for probs
cost = ctc_loss(probs, labels, probs_sizes, label_sizes)
cost.backward()

In [5]:
! git clone --recursive https://github.com/parlance/ctcdecode.git ; \
cd ctcdecode ; \
pip install .

Cloning into 'ctcdecode'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 722.80 KiB | 2.40 MiB/s, done.
Resolving deltas: 100% (485/485), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 9, done.        
remote: Counting objects: 100% (9/9), done.        
remote: Compressing objects: 100% (6/6), done.        
remote: Total 13264 (delta 3), reused 9 (delta 3), pack-reused 13255        
Receiving objects: 100% (13264/13264), 5.35 MiB | 9.14 MiB/s, done.


In [0]:
import ctcdecode

In [4]:
help(ctcdecode)

Help on package ctcdecode:

NAME
    ctcdecode

PACKAGE CONTENTS
    build
    ctcdecode (package)
    setup

FILE
    (built-in)


